In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Fetch page HTML

URL = "https://fabtcg.com/decklists/"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(URL, headers=headers)
response.status_code

200

In [ ]:
# Parse HTML

soup = BeautifulSoup(response.text, "html.parser")
soup.title.text

'Decklists Archive - Flesh and Blood TCG'

In [21]:
# Explore structure
 
tables = soup.find_all("table")
len(tables)

1

In [ ]:
# Extract rows incrementally

rows = tables[0].find_all("tr")
len(rows)

21

In [ ]:
# Inspect the table structure

rows = tables[0].find_all("tr")

# Print header row
headers = [th.get_text(strip=True) for th in rows[0].find_all("th")]
headers

['Country/Region', 'Date', 'Name', 'Event', 'Format', 'Hero', 'Rank']

In [ ]:
# Inspect a single data row

row = rows[1]
cells = row.find_all("td")

len(cells), [cell.get_text(strip=True) for cell in cells]


(7,
 ['Hong Kong',
  '21 Dec 2025',
  'Evanking Chiu – Chane – Battle Hardened: Hong Kong',
  'Battle Hardened: Hong Kong',
  'Silver Age',
  'Chane',
  '1st'])

In [22]:
# locate the decklist URL

links = row.find_all("a")
[(a.get_text(strip=True), a.get("href")) for a in links]


[('Evanking Chiu – Chane – Battle Hardened: Hong Kong',
  'https://fabtcg.com/decklists/evanking-chiu-chane-battle-hardened-hong-kong-2025/'),
 ('Battle Hardened: Hong Kong',
  'https://fabtcg.com/organised-play/2025/calling-hong-kong/')]

In [ ]:
# Build absolute deck URL 

from urllib.parse import urljoin

BASE_URL = "https://fabtcg.com"

deck_link = row.find("a")["href"]
urljoin(BASE_URL, deck_link)

'https://fabtcg.com/decklists/evanking-chiu-chane-battle-hardened-hong-kong-2025/'

In [24]:
# Extract one row into a dict

headers = headers  # from Cell 6|

data = {
    headers[i]: cells[i].get_text(strip=True)
    for i in range(len(cells))
}

data["deck_url"] = urljoin(BASE_URL, cells[2].find("a")["href"])
data


{'Country/Region': 'Hong Kong',
 'Date': '21 Dec 2025',
 'Name': 'Evanking Chiu – Chane – Battle Hardened: Hong Kong',
 'Event': 'Battle Hardened: Hong Kong',
 'Format': 'Silver Age',
 'Hero': 'Chane',
 'Rank': '1st',
 'deck_url': 'https://fabtcg.com/decklists/evanking-chiu-chane-battle-hardened-hong-kong-2025/'}

In [25]:

# Look al rows (preview only)

decklists = []

for row in rows[1:]:
    cells = row.find_all("td")
    if not cells:
        continue

    entry = {
        headers[i]: cells[i].get_text(strip=True)
        for i in range(len(cells))
    }

    link_tag = cells[2].find("a")
    entry["deck_url"] = urljoin(BASE_URL, link_tag["href"]) if link_tag else None

    decklists.append(entry)

len(decklists)


20

In [28]:
# Convert to DataFrame 

import pandas as pd

df = pd.DataFrame(decklists)
df.head()


,Country/Region,Date,Name,Event,Format,Hero,Rank,deck_url
0,Hong Kong,21 Dec 2025,Evanking Chiu – Chane – Battle Hardened: Hong ...,Battle Hardened: Hong Kong,Silver Age,Chane,1st,https://fabtcg.com/decklists/evanking-chiu-cha...
1,Philippines,21 Dec 2025,"Mikko Pena – Ira, Crimson Haze – Battle Harden...",Battle Hardened: Hong Kong,Silver Age,"Ira, Crimson Haze",2nd,https://fabtcg.com/decklists/mikko-pena-ira-cr...
2,Hong Kong,21 Dec 2025,erwin leung – Oldhim – Battle Hardened: Hong Kong,Battle Hardened: Hong Kong,Silver Age,Oldhim,3rd,https://fabtcg.com/decklists/erwin-leung-oldhi...
3,Hong Kong,21 Dec 2025,Isaac Lee – Chane – Battle Hardened: Hong Kong,Battle Hardened: Hong Kong,Silver Age,Chane,3rd,https://fabtcg.com/decklists/isaac-lee-chane-b...
4,Netherlands,21 Dec 2025,Paul van Gijssel – Chane – Battle Hardened: Ho...,Battle Hardened: Hong Kong,Silver Age,Chane,5th,https://fabtcg.com/decklists/paul-van-gijssel-...


In [33]:
# Inspect pagination links

links = soup.find_all("a", href=True)
[p["href"] for p in links if "/decklists/page/" in p["href"]][:10]

['https://fabtcg.com/decklists/page/2/',
 'https://fabtcg.com/decklists/page/184/',
 'https://fabtcg.com/decklists/page/2/']

In [ ]:
# Extract last page number safely

import re

page_numbers = set()

for a in soup.find_all("a", href=True):
    match = re.search(r"/decklists/page/(\d+)/", a["href"])
    if match:
        page_numbers.add(int(match.group(1)))

max(page_numbers)


184

In [35]:
# Generate all pages URLs

BASE_URL = "https://fabtcg.com"

page_urls = [
    f"{BASE_URL}/decklists/" if i == 1 else f"{BASE_URL}/decklists/page/{i}/"
    for i in range(1, max(page_numbers) + 1)
]

page_urls[:5], page_urls[-3:]


(['https://fabtcg.com/decklists/',
  'https://fabtcg.com/decklists/page/2/',
  'https://fabtcg.com/decklists/page/3/',
  'https://fabtcg.com/decklists/page/4/',
  'https://fabtcg.com/decklists/page/5/'],
 ['https://fabtcg.com/decklists/page/182/',
  'https://fabtcg.com/decklists/page/183/',
  'https://fabtcg.com/decklists/page/184/'])

In [36]:
# Reuse the Extraction Function (unchanged)

def extract_table_rows(soup, headers):
    table = soup.find("table")
    rows = table.find_all("tr")[1:]

    results = []

    for row in rows:
        cells = row.find_all("td")
        if not cells:
            continue

        entry = {
            headers[i]: cells[i].get_text(strip=True)
            for i in range(len(cells))
        }

        link_tag = cells[2].find("a")
        entry["deck_url"] = urljoin("https://fabtcg.com", link_tag["href"]) if link_tag else None

        results.append(entry)

    return results


In [37]:
# Fetch all pages

import requests
from bs4 import BeautifulSoup

HEADERS = {"User-Agent": "Mozilla/5.0"}

all_decklists = []

for url in page_urls:
    resp = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(resp.text, "html.parser")

    rows = extract_table_rows(soup, headers)
    all_decklists.extend(rows)

len(all_decklists)


3673

In [ ]:
# Final DataFrame
import pandas as pd

df_all = pd.DataFrame(all_decklists)
df_all.head(15)


,Country/Region,Date,Name,Event,Format,Hero,Rank,deck_url
0,Hong Kong,21 Dec 2025,Evanking Chiu – Chane – Battle Hardened: Hong ...,Battle Hardened: Hong Kong,Silver Age,Chane,1st,https://fabtcg.com/decklists/evanking-chiu-cha...
1,Philippines,21 Dec 2025,"Mikko Pena – Ira, Crimson Haze – Battle Harden...",Battle Hardened: Hong Kong,Silver Age,"Ira, Crimson Haze",2nd,https://fabtcg.com/decklists/mikko-pena-ira-cr...
2,Hong Kong,21 Dec 2025,erwin leung – Oldhim – Battle Hardened: Hong Kong,Battle Hardened: Hong Kong,Silver Age,Oldhim,3rd,https://fabtcg.com/decklists/erwin-leung-oldhi...
3,Hong Kong,21 Dec 2025,Isaac Lee – Chane – Battle Hardened: Hong Kong,Battle Hardened: Hong Kong,Silver Age,Chane,3rd,https://fabtcg.com/decklists/isaac-lee-chane-b...
4,Netherlands,21 Dec 2025,Paul van Gijssel – Chane – Battle Hardened: Ho...,Battle Hardened: Hong Kong,Silver Age,Chane,5th,https://fabtcg.com/decklists/paul-van-gijssel-...
